In [1]:
def DoubleCircuits(M,IngletonMain=False):
    #input: a matroid M
    #       a boolean indicating whether Ingleton-Main is checked or not (then Dress-Lovasz is checked)
    #output: for each double circuit CC:
    #          the double circuit degree d, 
    #          the number of points to be added to the intersection
    #          the circuit closures of the double circuit
    #          the intersection of these circuits
    Md = M.dual()
    for F in Md.flats(Md.full_rank() - 2):
        Mdm = Md.contract(F)
        d=Mdm.simplify().size()
        if IngletonMain:
            goodDegree = (d == 3)
        else:
            goodDegree = (d >= 3)
        if goodDegree:
            circs = [M.closure(C) for C in Mdm.cocircuits()]
            inter = M.groundset()
            for C in circs:
                inter = inter.intersection(C)
            ir = M.rank(inter)
            if d-2 - ir <= 0:
                continue
            yield d, circs, inter

In [2]:
def DressLovaszExtensions(M, CC):
    #input: a matroid M
    #       a list of circuit closures CC
    #output: all matroid extensions of M by i such that i lies in all circuit closures of CC
    E = M.groundset()
    i = -1
    goodExt = True
    while i in E:
        i -= 1
    
    return M.extensions(element=i,subsets=[list(c) for c in CC if len(c) < M.size()]), i

def RecursiveDressLovaszCondition(M,depth=1,IngletonMain=False):
    #input: a matroid M
    #       an integer depth
    #       a boolean indicating whether Ingleton-Main is checked or not (then Dress-Lovasz is checked)
    #output: the Dress-Lovasz (or Ingleton-Main) condition at depth 'depth' for M
    
    if depth==0:
        return True
    
    it = 0
    for dCC in DoubleCircuits(M,IngletonMain):
        d, CC, inter = dCC
        DLE, i = DressLovaszExtensions(M,CC)
        goodCC = False
        for N in DLE:
            if N.rank(inter.union([i])) == N.rank(inter):
                continue
            if RecursiveDressLovaszCondition(N,depth-1):
                goodCC = True
                break
        if not goodCC:
            return False
    return True

In [3]:
M = matroids.named_matroids.TicTacToe()
Md = M.dual()

In [4]:
RecursiveDressLovaszCondition(M)

True

In [9]:
len([CC for CC in DoubleCircuits(M)])

0

In [10]:
%%time
RecursiveDressLovaszCondition(Md)

CPU times: user 160 ms, sys: 0 ns, total: 160 ms
Wall time: 167 ms


True

In [11]:
%%time
RecursiveDressLovaszCondition(Md,3)

CPU times: user 641 ms, sys: 0 ns, total: 641 ms
Wall time: 651 ms


False